In [40]:
import refinitiv.dataplatform as rdp
import ipywidgets as widgets
import pandas as pd
import ipysheet
from datetime import datetime, timedelta

In [41]:
import eikon as ek
# The key is placed in a text file so that it may be used in this code without showing it itself:
eikon_key = open("eikon.txt", "r")
ek.set_app_key(str(eikon_key.read()))
# It is best to close the files we opened in order to make sure that we don't stop any other services/programs from accessing them if they need to:
eikon_key.close()

# ## If in Codebook:
# rdp.open_desktop_session('DEFAULT_CODE_BOOK_APP_KEY')
# ek = rdp.legacy

2021-09-03 17:56:21,807 P[20348] [MainThread 14792] Exception in callback _ProactorBasePipeTransport._call_connection_lost(None)
handle: <Handle _ProactorBasePipeTransport._call_connection_lost(None)>
Traceback (most recent call last):
  File "c:\users\u6082174.ten\appdata\local\programs\python\python38-32\lib\site-packages\nest_asyncio.py", line 150, in run
    ctx.run(self._callback, *self._args)
  File "c:\users\u6082174.ten\appdata\local\programs\python\python38-32\lib\asyncio\proactor_events.py", line 162, in _call_connection_lost
    self._sock.shutdown(socket.SHUT_RDWR)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host


In [42]:
ccy = ['SGD']
depo_tenors = ['ON', '1W', '1M', '3M', '6M', '1Y']
inst_dict = {'depo':'D=', 'fx_forward':'=', 'fx_fwd_outright':'V='}
fields_list = ['GV4_TEXT', 'BID', 'ASK']

In [43]:
ipycols = []
fields = fields_list
currency = ccy
inst_type = 'fx_forward'
custom_column_names = ['Tenor', 'BID', 'ASK']
tenors = depo_tenors
tenors

['ON', '1W', '1M', '3M', '6M', '1Y']

In [44]:
suffix = inst_dict[inst_type]
suffix

'='

In [45]:
init_port = list(map(lambda t: '%s%s%s' % (currency[0], t, suffix), tenors))  # same as ' [f"{currency[0]}{i}{suffix}" for i in tenors] '
init_port

['SGDON=', 'SGD1W=', 'SGD1M=', 'SGD3M=', 'SGD6M=', 'SGD1Y=']

In [46]:
instrument_name = init_port[0]
instrument_name

'SGDON='

In [47]:
dict_port = dict(zip(list(init_port), range(len(init_port))))
dict_port

{'SGDON=': 0, 'SGD1W=': 1, 'SGD1M=': 2, 'SGD3M=': 3, 'SGD6M=': 4, 'SGD1Y=': 5}

In [48]:
_df, err = ek.get_data(init_port, fields)
_df

,Instrument,GV4_TEXT,BID,ASK
0,SGDON=,ON,0.23,0.33
1,SGD1W=,<NA>,<NA>,<NA>
2,SGD1M=,1M,0.8,1.0
3,SGD3M=,3M,2.27,2.47
4,SGD6M=,6M,4.55,4.95
5,SGD1Y=,1Y,7.0,8.0


In [49]:
df = _df.drop(columns='Instrument').dropna()
df

,GV4_TEXT,BID,ASK
0,ON,0.23,0.33
2,1M,0.8,1.0
3,3M,2.27,2.47
4,6M,4.55,4.95
5,1Y,7.0,8.0


In [50]:
if len(df.columns) == len(custom_column_names):
    df.columns = custom_column_names
df

,Tenor,BID,ASK
0,ON,0.23,0.33
2,1M,0.8,1.0
3,3M,2.27,2.47
4,6M,4.55,4.95
5,1Y,7.0,8.0


In [51]:
sheet = ipysheet.sheet(columns=len(df.columns.tolist()),
                       rows=len(init_port),
                       column_headers=df.columns.tolist(),
                       row_headers=False)

In [52]:
for n, i in enumerate(df.columns):
    temp_col = ipysheet.column(n, df[i].tolist(), row_start=0, read_only=False)
    ipycols.append(temp_col)

In [53]:
sheet

Sheet(cells=(Cell(column_end=0, column_start=0, row_end=4, row_start=0, squeeze_row=False, type='text', value=…

In [54]:
sheet_columns = dict(zip(df.columns.tolist(), ipycols))
global_cols = sheet_columns

In [55]:
global_cols

{'Tenor': Cell(column_end=0, column_start=0, row_end=4, row_start=0, squeeze_row=False, type='text', value=['ON', '1M', '3M', '6M', '1Y']),
 'BID': Cell(column_end=1, column_start=1, row_end=4, row_start=0, squeeze_row=False, type='numeric', value=[0.23, 0.8, 2.27, 4.55, 7.0]),
 'ASK': Cell(column_end=2, column_start=2, row_end=4, row_start=0, squeeze_row=False, type='numeric', value=[0.33, 1.0, 2.47, 4.95, 8.0])}

In [56]:
r = init_port.index(instrument_name)
r

0

In [72]:
_df[['Instrument', 'BID', 'ASK']]

,Instrument,BID,ASK
0,SGDON=,0.23,0.33
1,SGD1W=,<NA>,<NA>
2,SGD1M=,0.8,1.0
3,SGD3M=,2.27,2.47
4,SGD6M=,4.55,4.95
5,SGD1Y=,7.0,8.0


In [38]:
for fld_name, fld_val in fields.items():
    c = self.fields.index(fld_name)
    print(c)
#     temp_column = list(global_cols[list(global_cols.keys())[c]].value)
#     temp_column[r] = fld_val
#     self.sheet_columns[list(global_cols.keys())[c]].value = list(temp_column)

AttributeError: 'list' object has no attribute 'items'

In [ ]:
class streaming_spreadsheet:
    
    def create_spreadsheet(self, currency, tenors, inst_type, fields, custom_column_names, add_change, change_time_frame):
        ipycols = []
        suffix = inst_dict[inst_type]
        init_port = list(map(lambda t: '%s%s%s' % (currency[0], t, suffix), tenors))  # same as ' [f"{currency[0]}{i}{suffix}" for i in tenors] '
        dict_port = dict(zip(list(init_port), range(len(init_port))))
        df, err = ek.get_data(init_port, fields)
        self.instruments = init_port
        df = df.drop(columns='Instrument')
        if add_change: 
            df['Change'] = 0
            #date = datetime.now() - timedelta(days=7)
            #to be completed with ek.get_timeseries()

        if len(df.columns) == len(custom_column_names):
            df.columns = custom_column_names
        sheet = ipysheet.sheet(columns= len(df.columns.tolist()), rows = len(init_port), column_headers=df.columns.tolist(), row_headers=False)
        for n, i in enumerate(df.columns):
            temp_col = ipysheet.column(n, df[i].tolist(), row_start=0, read_only=False)
            ipycols.append(temp_col)

        self.sheet_columns = dict(zip(df.columns.tolist(), ipycols))

        self.open_stream(init_port, fields)
        self.streaming_prices.open()
        return sheet

    def update_columns(self, init_port, streaming_prices, instrument_name, fields):
        global_cols = self.sheet_columns
        r = init_port.index(instrument_name)
        for fld_name, fld_val in fields.items():
            c = self.fields.index(fld_name)
            temp_column = list(global_cols[list(global_cols.keys())[c]].value)
            temp_column[r] = fld_val
            self.sheet_columns[list(global_cols.keys())[c]].value = list(temp_column)
        return

    def open_stream(self, init_port, fields):
        self.streaming_prices = rdp.StreamingPrices(
            universe = init_port, 
            fields = fields,
            on_update = lambda streaming_prices, instrument_name, fields: 
                self.update_columns(init_port, streaming_prices, instrument_name, fields)
        )
        return

    def close_stream(self):
        self.streaming_prices.close()
        return
    
    def __init__(self, currency, tenors, inst_type, fields, custom_column_names, add_change=False, change_time_frame=''):
        self.sheet_columns = {}
        self.instruments = []
        self.streaming_price = 0
        self.currency = currency
        self.tenors = tenors
        self.inst_type = inst_type
        self.fields = fields
        self.custom_column_names = custom_column_names
        self.add_change = add_change
        self.change_time_frame = change_time_frame
        self.sheet = self.create_spreadsheet(currency, tenors, inst_type, fields, custom_column_names, add_change, change_time_frame)
        